In [ ]:
from imgcls.classification.yolov8 import YoloUltralyticsPipeline, plot_image_seg
from imgcls.io import *

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Custom pipeline for running yolov8 model for image classification
- Seealso documentation in module `imgcls.classification.yolov8.pipeline.YoloUltralyticsPipeline`
- Check https://docs.ultralytics.com/modes/train/
- Follow the data folder structure
    - xx 

In [ ]:
# Init the class. REPLACE `CACHE_DIRECTORY` to user-specific root directory which followed by the data folder structure
# By default, assume the dataset is located at ~/.cache/comvis/imgcls from home directory
yolo = YoloUltralyticsPipeline(root_dir=CACHE_DIRECTORY, resize_dim=(500, 500))

## **Step 1** - Clone batch raw .npy files to png in a separated folder

In [ ]:
yolo.clone_png_dir()

In [ ]:
# see example by indices
plot_image_seg(yolo.image_dir, index_range=(0, 5))

## **Step 2** - Generate the yaml for yolov8 config

In [ ]:
# <PATH> related values are user-specific
yolo.gen_yaml(verbose=True)

## **Step 3** - Detect the object edge from seg files and generate the yolo4 required label file for train dataset
    - <class_id> <xc> <y_center> <width> <height>

In [ ]:
# Use debug mode = True to see the examples 
yolo.gen_label_txt(debug_mode=True)